In [35]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir='pt_logs')
# Load the dataset and perform exploratory data analysis
# from sklearn.datasets import load_digits
# from sklearn.model_selection import train_test_split
# digits = load_digits()


In [36]:
# Plot some examples of the images
# for i in range(2):
#     plt.imshow(digits.images[i], cmap='gray')
#     plt.title(f"Label: {digits.target[i]}")
#     plt.show()


In [37]:
# # Prepare the data for training
# X = digits.images.reshape(-1, 64)
# y = digits.target
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
# test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [38]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_ds = datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
test_ds = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
trainLoader = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)
testLoader = torch.utils.data.DataLoader(test_ds, batch_size=32, shuffle=True)

Files already downloaded and verified


In [39]:
# Define the model
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32*32*3,1536)
        self.fc2 = nn.Linear(1536,32)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [40]:
# Train the model
model = MLP()
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(trainLoader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        inputs = inputs.view(-1,32*32*3)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        writer.add_scalar('Loss/train', loss.item(), epoch)
    # print(data[0][0][0])
    writer.add_graph(model,data[0].view(-1,32*32*3))


In [46]:

# Evaluate the model on the test data
correct = 0
total = len(data)
with torch.no_grad():
    for data in testLoader:
        inputs, labels = data
        grid = torchvision.utils.make_grid(inputs)
        writer.add_image('image',grid,0)
        inputs = inputs.view(-1,32*32*3)
        outputs = model(inputs)
        _, preds = torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+=(preds == labels).sum().item()
print((correct/total)*100)
# writer.add_scalar('Accuracy', 100*(correct/total))


50.929814037192564


In [45]:
%load_ext tensorboard
# %reload_ext tensorboard
log_folder = './pt_logs'
%tensorboard --logdir={log_folder}